# Create an InterActive Graph of NTRL Lab Analysis Results

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [229]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------


In [230]:
# store the Filepath of the shutdown schedule excel file
#shutdown_filepath = r'\\THGOBNA002\thpal\05 Production\260 Tech - Shutdown\Shutdown 202009 Major Shutdown\Process B\00 Schedule\2020 Major SD Schedule Rev 2.4.xlsx'
shutdown_filepath2 = r'C:\Users\v.t.flores\Documents\2020 Major SD Progress Manual Update.xlsx'

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
# Load the shutdown schedule excel file
raw_shutdown_sched = pd.read_excel(shutdown_filepath2, sheet_name='Sheet1', index_col=False)


#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
# CLEAN UP THE 2020 Major SD Progress Manual Update File DF
raw_shutdown_sched = raw_shutdown_sched.drop(raw_shutdown_sched.index[0:2])
shutdown_sched_1 = raw_shutdown_sched[['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']].iloc[57:]  ## SELECT THE ROWS AND COLUMNS NEEDED
shutdown_sched_1.columns = ['Equipment', 'Activities','START_plan','FINISH_plan',"Duration_hrs_plan", 'START_actual','FINISH_actual',"Duration_hrs_ETA"] ## RENAME THE COLUMNS

shutdown_sched_1 = shutdown_sched_1.reset_index().drop('index', axis=1)
shutdown_sched_1['Equipment'] = shutdown_sched_1['Equipment'].fillna(method='ffill') ## FILL THE BLANK CELLS IN THE EQUIPMENT COLUMN USING THE FFILL METHOD.

index_list = shutdown_sched_1[pd.isna(shutdown_sched_1['START_plan'])].index # takes the index of the column 'START_plan' with Nan values and stores it in the variable index_list
shutdown_sched_2 = shutdown_sched_1.drop(index_list) # Drop the nan values found in the 'START_plan' column then store to a new df called shutdown_sched_2

shutdown_sched_2['DATETIME'] = shutdown_sched_2['START_plan'] ## CREATE A DATETIME COLUMN EQUAL TO THE START COLUMN.
shutdown_sched_3 = shutdown_sched_2.set_index('DATETIME') ## SET THE INDEX TO DATETIME. THIS TELLS US WHEN THE ACTIVITY IS SCHEDULED TO START.

nan_index_list = shutdown_sched_3[pd.isna(shutdown_sched_3['Duration_hrs_plan'])].index # find nan values in the 'Duration_hrs_plan' column
shutdown_sched_3 = shutdown_sched_3.drop(nan_index_list) # Drop the nan values found in the 'Duration_hrs_plan'


In [231]:
# this code block computes the %progress per activity and the necessary columns needed to compute the % progress_actual

shutdown_sched_3['Reference_time'] = np.where(shutdown_sched_3['FINISH_actual'].isnull(), datetime.datetime.now(), shutdown_sched_3['FINISH_actual']) # column 9
shutdown_sched_3['Reference_time'] = [parse(str(ref_time)) for ref_time in shutdown_sched_3['Reference_time']]

shutdown_sched_3['START_plan'] = [parse(str(timestart)) for timestart in shutdown_sched_3['START_plan']]

shutdown_sched_3['Duration_hrs_plan'] = [timedelta(hours=x) for x in shutdown_sched_3['Duration_hrs_plan']] # column 5

shutdown_sched_3['hours_since_start'] = np.where(shutdown_sched_3['START_plan'].notnull(), 
                                                 shutdown_sched_3['Reference_time'] - shutdown_sched_3['START_plan'], 
                                                 0) #compute the hours since the start of the activity. column 10 

shutdown_sched_3['Duration_hrs_ETA'] = np.where(shutdown_sched_3['Duration_hrs_plan'] < shutdown_sched_3['hours_since_start'], 
                                                shutdown_sched_3['hours_since_start'] + timedelta(hours=1), 
                                                shutdown_sched_3['Duration_hrs_plan']) # column 11

reference_time_now = datetime.datetime.now()
shutdown_sched_3['{}'.format(reference_time_now)] = 100*shutdown_sched_3['hours_since_start']/shutdown_sched_3['Duration_hrs_ETA'] # column 12 onwards
shutdown_sched_3['{}'.format(reference_time_now)] = np.where()


In [261]:
shutdown_sched_3

,Equipment,Activities,START_plan,FINISH_plan,Duration_hrs_plan,START_actual,FINISH_actual,Duration_hrs_ETA,Reference_time,hours_since_start,2020-10-15 16:18:41.420881
DATETIME,,,,,,,,,,,
2020-10-22 00:00:00,De-Zn Feedline,104PU01 Stop,2020-10-22 00:00:00,2020-10-22 01:00:00,0 days 01:00:00,2020-10-22 01:00:00,2020-10-22 02:00:00,0 days 03:00:00,2020-10-22 02:00:00.000000,0 days 02:00:00,66.666667
2020-10-22 01:00:00,De-Zn Feedline,LOTO Installation (Valves & Pump),2020-10-22 01:00:00,2020-10-22 03:00:00,0 days 02:00:00,2020-10-22 02:00:00,2020-10-22 05:00:00,0 days 05:00:00,2020-10-22 05:00:00.000000,0 days 04:00:00,80.000000
2020-10-22 01:00:00,De-Zn Feedline,Feedline level lowering/105TK01 Inerting,2020-10-22 01:00:00,2020-10-22 10:00:00,0 days 09:00:00,2020-10-22 02:00:00,2020-10-22 12:00:00,0 days 12:00:00,2020-10-22 12:00:00.000000,0 days 11:00:00,91.666667
2020-10-22 10:00:00,De-Zn Feedline,Feed Line Blinding/105TK01 inlet,2020-10-22 10:00:00,2020-10-22 12:00:00,0 days 02:00:00,2020-10-22 12:00:00,2020-10-22 14:00:00,0 days 05:00:00,2020-10-22 14:00:00.000000,0 days 04:00:00,80.000000
2020-10-22 12:00:00,De-Zn Feedline,Feed Line Draining,2020-10-22 12:00:00,2020-10-22 20:00:00,0 days 08:00:00,2020-10-22 14:00:00,2020-10-22 20:00:00,0 days 08:00:00,2020-10-22 20:00:00.000000,0 days 08:00:00,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
2020-10-31 17:00:00,530CT31,Conditioning (Chemical dosing with heat load),2020-10-31 17:00:00,2020-11-01 17:00:00,1 days 00:00:00,NaN,NaN,1 days 00:00:00,2020-10-15 16:18:41.316768,-17 days +23:18:41.316768,0.000000
2020-11-02 08:00:00,525NG01,525CP01AB PMS Activity (1 compressor/ day),2020-11-02 08:00:00,2020-11-05 08:00:00,3 days 00:00:00,NaN,NaN,3 days 00:00:00,2020-10-15 16:18:41.316768,-18 days +08:18:41.316768,0.000000
2020-11-02 08:00:00,525NG01,KV valves inspection,2020-11-02 08:00:00,2020-11-06 08:00:00,4 days 00:00:00,NaN,NaN,4 days 00:00:00,2020-10-15 16:18:41.316768,-18 days +08:18:41.316768,0.000000


In [233]:
# CREATE A LIST CONTAINING ALL UNIQUE ENTRIES IN THE EQUIPMENT COLUMN
List_equiments = shutdown_sched_3['Equipment'].unique()
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column


In [ ]:
# replace the values in the df_prcnt_progress_transposed to a range 0 to 100
for col in shutdown_sched_3.columns[10:]:
    print(col)
    shutdown_sched_3[col] = np.where(shutdown_sched_3[col]>=100, 100, 
                                     np.where(shutdown_sched_3[col]<0, 0, shutdown_sched_3[col]))

In [277]:
shutdown_sched_3[['Reference_time', 'Equipment']]

,Reference_time,Equipment
DATETIME,,
2020-10-22 00:00:00,2020-10-22 02:00:00.000000,De-Zn Feedline
2020-10-22 01:00:00,2020-10-22 05:00:00.000000,De-Zn Feedline
2020-10-22 01:00:00,2020-10-22 12:00:00.000000,De-Zn Feedline
2020-10-22 10:00:00,2020-10-22 14:00:00.000000,De-Zn Feedline
2020-10-22 12:00:00,2020-10-22 20:00:00.000000,De-Zn Feedline
...,...,...
2020-10-31 17:00:00,2020-10-15 16:18:41.316768,530CT31
2020-11-02 08:00:00,2020-10-15 16:18:41.316768,525NG01
2020-11-02 08:00:00,2020-10-15 16:18:41.316768,525NG01


In [282]:
shutdown_sched_4 = shutdown_sched_3.groupby(['Equipment', 'Reference_time'])[shutdown_sched_3.columns[10:]].mean().T

In [264]:
df_equipment_Grouped = shutdown_sched_4.groupby('Equipment')[shutdown_sched_3.columns[10:]].mean().T

In [265]:
df_equipment_Grouped

Equipment,105FT01A,105FT01B,105FT01C,105FT01D,105FT01E,105FT01F,105PU01ABCDEFG,105SP01/02,105TK01,105TK02,...,114TK02,115TK01,115TK02,"24"" SPT tie in",525NG01,530CT31,De-Zn Feedline,MS Common Vent/ 106SP01,MS FEEDLINE,VSLC RHL Line
2020-10-15 16:18:41.420881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,80.07716,0.0,0.0,0.0


In [253]:
# prepare a list of columns for use in summing up progress per area    
columns_dezn = ['De-Zn Feedline', '105TK01', '105TK02', '105TK03', '105TK05',
                '105SP01/02', '105PU01ABCDEFG', '105FT01A', '105FT01B', '105FT01C',
                '105FT01D', '105FT01E', '105FT01F', '105TK13', '105TK08']

columns_MS = ['106TK01', 'MS FEEDLINE', '106VE01', '106VE02', '106VE03', 
              '106VE04', '106VE05', '106HX02', '106VB01AB', '106CP01AB', 
              '106TH01', '106FT02A', '106FT02B', '106TK08', '106TK04',
              '106TK05', '106TK18', '106TK14', '106SP02', 'MS Common Vent/ 106SP01',
              '106SR00/02\n106SP03/04', '106SR03', '106SR04', '106TK15/ NaOH header line']

columns_108_chem = [ '108TK01', '108TK04','24" SPT tie in', 'VSLC RHL Line', '112TK00',
                    '112TK01', '114HP/FD01', '114TK01', '114TK02', '114 Pumps', 
                    '115TK01','115TK02']
columns_111_ls = ['111 Belt Conveyors\n111VS01\n111CR01AB', '111ML01','111TK01', '111PU01/\n111CY01', '111TK02', 
                  '530CT31', '525NG01']

In [285]:
shutdown_sched_3['Equipment']

,Equipment,Activities,START_plan,FINISH_plan,Duration_hrs_plan,START_actual,FINISH_actual,Duration_hrs_ETA,Reference_time,hours_since_start,2020-10-15 16:18:41.420881
DATETIME,,,,,,,,,,,
2020-10-22 00:00:00,De-Zn Feedline,104PU01 Stop,2020-10-22 00:00:00,2020-10-22 01:00:00,0 days 01:00:00,2020-10-22 01:00:00,2020-10-22 02:00:00,0 days 03:00:00,2020-10-22 02:00:00.000000,0 days 02:00:00,66.666667
2020-10-22 01:00:00,De-Zn Feedline,LOTO Installation (Valves & Pump),2020-10-22 01:00:00,2020-10-22 03:00:00,0 days 02:00:00,2020-10-22 02:00:00,2020-10-22 05:00:00,0 days 05:00:00,2020-10-22 05:00:00.000000,0 days 04:00:00,80.000000
2020-10-22 01:00:00,De-Zn Feedline,Feedline level lowering/105TK01 Inerting,2020-10-22 01:00:00,2020-10-22 10:00:00,0 days 09:00:00,2020-10-22 02:00:00,2020-10-22 12:00:00,0 days 12:00:00,2020-10-22 12:00:00.000000,0 days 11:00:00,91.666667
2020-10-22 10:00:00,De-Zn Feedline,Feed Line Blinding/105TK01 inlet,2020-10-22 10:00:00,2020-10-22 12:00:00,0 days 02:00:00,2020-10-22 12:00:00,2020-10-22 14:00:00,0 days 05:00:00,2020-10-22 14:00:00.000000,0 days 04:00:00,80.000000
2020-10-22 12:00:00,De-Zn Feedline,Feed Line Draining,2020-10-22 12:00:00,2020-10-22 20:00:00,0 days 08:00:00,2020-10-22 14:00:00,2020-10-22 20:00:00,0 days 08:00:00,2020-10-22 20:00:00.000000,0 days 08:00:00,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
2020-10-31 17:00:00,530CT31,Conditioning (Chemical dosing with heat load),2020-10-31 17:00:00,2020-11-01 17:00:00,1 days 00:00:00,NaN,NaN,1 days 00:00:00,2020-10-15 16:18:41.316768,-17 days +23:18:41.316768,0.000000
2020-11-02 08:00:00,525NG01,525CP01AB PMS Activity (1 compressor/ day),2020-11-02 08:00:00,2020-11-05 08:00:00,3 days 00:00:00,NaN,NaN,3 days 00:00:00,2020-10-15 16:18:41.316768,-18 days +08:18:41.316768,0.000000
2020-11-02 08:00:00,525NG01,KV valves inspection,2020-11-02 08:00:00,2020-11-06 08:00:00,4 days 00:00:00,NaN,NaN,4 days 00:00:00,2020-10-15 16:18:41.316768,-18 days +08:18:41.316768,0.000000


In [257]:

# Compute the average %progress per area
actual_MS_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_MS], axis=1)
actual_DeZn_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_dezn], axis=1)
actual_FNTRL_chem_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_108_chem], axis=1)
actual_LS_CT_progress_plan_MAINTE = np.mean(df_equipment_Grouped[columns_111_ls], axis=1)

# Create a column for the %progress values per area
df_equipment_Grouped['MS_progress_plan'] = actual_MS_progress_plan_MAINTE
df_equipment_Grouped['DeZn_progress_plan'] = actual_DeZn_progress_plan_MAINTE
df_equipment_Grouped['FNTRL_chem_progress_plan'] = actual_FNTRL_chem_progress_plan_MAINTE
df_equipment_Grouped['LS_CT_progress_plan'] = actual_LS_CT_progress_plan_MAINTE






In [259]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

#### Create the Plot

In [258]:
#### Plotting the SD progress including the maintenance activities
#
fig_SD_MAINTE_actual = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "scatter"}]])
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['MS_progress_plan'].index, y=df_equipment_Grouped['MS_progress_plan'], name='MS_progress_plan'), secondary_y=False,)
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['DeZn_progress_plan'].index, y=df_equipment_Grouped['DeZn_progress_plan'], name='DeZn_progress_plan'), secondary_y=False,)     
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['FNTRL_chem_progress_plan'].index, y=df_equipment_Grouped['FNTRL_chem_progress_plan'], name='FNTRL_chem_progress_plan'), secondary_y=False,)    
fig_SD_MAINTE_actual.add_trace(go.Scatter(x=df_equipment_Grouped['LS_CT_progress_plan'].index, y=df_equipment_Grouped['LS_CT_progress_plan'], name='LS_CT_progress_plan'), secondary_y=False,)    
# adding a range slider below the graph in plotly
#Example:
#title_plotAcid = 'SD Progress 2020'
#fig_SD.update_layout(title_text=title_plotAcid,xaxis=dict(rangeslider=dict(visible = True),type='date'))
fig_SD_MAINTE_actual.update_xaxes(title_text="Datetime") # Set x-axis title
fig_SD_MAINTE_actual.update_yaxes(title_text="ShutDown Progress %",secondary_y=False) # Set y-axes titles primary axis


title_plot_SD = "<b>MS PLANT Shutdown Progress</b> Hourly Tracker"
fig_SD_MAINTE_actual.update_layout(title_text=title_plot_SD ,hovermode="x",
                       hoverdistance=100, # Distance to show hover label of data point
                       spikedistance=1000, # Distance to show spike
                       xaxis=dict(linecolor="#BCCCDC",
                                   showspikes=True, # Show spike line for X-axis
                                   # Format spike
                                   spikethickness=2,
                                   spikedash="dot",
                                   spikecolor="#999999",
                                   spikemode="across"), font=dict(size=10)
                               )

# Mark the datetime of power limitation by shading these regions with light color
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-01 04:30:00',
            y0=0,
            x1='2020-11-01 19:00:00',
            y1=100,
            fillcolor="pink",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-08 16:00:00',
            y0=0,
            x1='2020-11-09 18:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-10-23 14:00:00',
            y0=0,
            x1='2020-10-25 08:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig_SD_MAINTE_actual.update_shapes(dict(xref='x', yref='y'))

# Mark the datetime of power limitation by shading these regions with light color
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-01 04:30:00',
            y0=0,
            x1='2020-11-01 19:00:00',
            y1=100,
            fillcolor="pink",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-08 16:00:00',
            y0=0,
            x1='2020-11-09 18:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

fig_SD_MAINTE_actual.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-10-23 14:00:00',
            y0=0,
            x1='2020-10-25 08:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig_SD_MAINTE_actual.update_shapes(dict(xref='x', yref='y'))

fig_SD_MAINTE_actual.add_annotation(
        x='2020-10-24 10:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SD_MAINTE_actual.add_annotation(
        x='2020-11-09 05:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SD_MAINTE_actual.add_annotation(
        x='2020-11-01 12:00:00',
        xref="x",
        yref="paper",
        text="E&I Blackout Activity",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#850017",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

#### Export the chart as HTML

In [ ]:
plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\MS_Progress_of_Equipment_Handover_Tracker.html', 'w') as f:
    f.write(fig_SD_MAINTE.to_html(fig_SD_MAINTE, full_html=True, include_plotlyjs='cdn'))
    


#### Temporary Code

In [ ]:
# TEMPORARY CODE FOR GETTING THE ACTIVITIES PER EQUIPMENT PER AREA
"""
progress_plan_columns = shutdown_sched_4.columns[-504:] # the list of columns containing the %progress per activity as planned

List_equiments = shutdown_sched_4['Equipment'].unique() # list of the unique activities in the shutdown_sched_4 dataframe
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column

# Initiallize the variables with zeros
SUM_percent_progress_of_activity_array = 0 #np.zeros((1, len(progress_plan_columns))) # the array containing all the %progress for a single equipment for the entire duration of the shutdown
SUM_percent_progress_of_equipment_array = np.zeros((len(List_equiments), len(progress_plan_columns))) # the array containing all the %progress for all the equipments for the entire duration of the shutdown


for eqpmnt_index in range(len(List_equiments_rshp)): #loop through the list of equipments
    eq = List_equiments_rshp[eqpmnt_index][0] # get a single name of the equipment from the list of equipments
    print(eq) # Print the equipment taken the list and check it in the output for confirmation
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq)].count() # this counts the total number of activities under the same equipment. this includes the maintenance activities
    
    total_activities_count = 0 # initiallize the total_activities counter to zero. this counts all the activities prior to maintenance handover or inspection. so this counts the prodn activts leading up to handover
    percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of activity progress for the entire shutdown daterange
    SUM_percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of the percent progress of all the activities for the same equipment for the entire shutdown daterange
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])): # loop thru all the rows with the same equipment name eq
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]   # get the activity name one by one
        percent_progress_of_activity_array = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq][progress_plan_columns].iloc[act]   # get the entire array of the progress for entire SD daterange
        # np.array() = activity_names
        # percent_progress_
        
        SUM_percent_progress_of_activity_array = SUM_percent_progress_of_activity_array + percent_progress_of_activity_array # add up all the progress of all the activities for the same equipment

        print(activity_name)
        match = re.search(r'\binspection\b',activity_name.lower()) # check the activity for the keyword 'inspection'
        match1 = re.search(r'\bmaintenance\b',activity_name.lower()) # check the activity for the keyword 'maintenance'
        if (not match) and (not match1) : # check if there is no match
            #print(activity_name)
            total_activities_count+=1
        else:
            print("index: {}, activitiy: {}".format(total_activities_count, activity_name))
            break # stop looping on the same equipment if the activity matches the keywords 'inspection' or 'maintenance'
    
    equipment_percent_progress_total = 100*total_activities_count
    SUM_percent_progress_of_equipment_array[eqpmnt_index] = 100*SUM_percent_progress_of_activity_array/equipment_percent_progress_total
       
df_prcnt_progress = pd.DataFrame(SUM_percent_progress_of_equipment_array)
df_prcnt_progress['equipment'] = List_equiments_rshp

"""
# END OF TEMPORARY CODE

#### End of Temporary code

### Annex codes

In [ ]:
# Spare code
# DO NOT EXECUTE. This code block is for continuation
# This code block is non vectorized.
# For use in handling the input data for the actual progress which takes in the datetime when the activity is completed. DCS controller inputs this data manually.
#
#
"""
reference_datetime = parse('2020-10-31 02:00:00')#datetime.datetime.now()

# Initiallize the variables with zeros

equipment_percent_complte = np.zeros((len(List_equiments_rshp),1))

activities_arr = np.zeros((len(shutdown_sched_4['Activities']),1))
percent_prog_act_Plan_arr = np.zeros((len(shutdown_sched_4['Activities']),1))
reference_datetime_arr = np.zeros((len(shutdown_sched_4['Activities']),1))

for eqpmnt_index in range(len(List_equiments_rshp)):
    eq = List_equiments_rshp[eqpmnt_index][0]
    print(eq)
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq) & (shutdown_sched_4['Complete_or_Not'] == 'Completed')].count()['Complete_or_Not']
    
    #total_act_per_eqpmnt = len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])
    total_activities_count = 0
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])):
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]
        time_start = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['START_plan'].iloc[act]
        duration = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Duration_hrs_plan'].iloc[act]
        
        #Calculate the time difference in hours
        print("time start: {}, reference time: {}".format(time_start, reference_datetime))
        diff = reference_datetime - parse(str(time_start))
        diff_hours = diff.days*24 + diff.seconds/3600
        
        #(datetime_diff/duration).days/24 + (datetime_diff/duration).seconds/3600
        print(diff_hours)
        # Calculate the planned progress per activity
        #
        # 
        # Compute the planned shutdown progress
        if duration != 0: # check if the duration is not set to zero
        
            planned_progress_per_act = 100*diff_hours/duration
            if planned_progress_per_act > 0:
                if planned_progress_per_act >= 100:
                    planned_progress_per_act = 100
                    print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                    percent_prog_act_Plan_arr[act] = round(planned_progress_per_act, 2) #store the calculated percent progress into the array called percent_prog_act_Plan_arr
                elif planned_progress_per_act < 100:
                    print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                    percent_prog_act_Plan_arr[act] = round(planned_progress_per_act, 2) #store the calculated percent progress into the array called percent_prog_act_Plan_arr
                    
            else:
                planned_progress_per_act = 0
                print("ref_time: {}, time start {}, activity: {}, %Progress {} %, duration:{}, index: {}".format(reference_datetime, time_start, activity_name, round(planned_progress_per_act, 2), duration, act))
                percent_prog_act_Plan_arr[act] = planned_progress_per_act #store the calculated percent progress into the array called percent_prog_act_Plan_arr
        else:
            planned_progress_per_act = 0
            percent_prog_act_Plan_arr[act] = planned_progress_per_act #store the calculated percent progress into the array called percent_prog_act_Plan_arr
        #
        # End code for the percent progress calculation
        
        
        match = re.search(r'\binspection\b',activity_name.lower())
        if not match:
            print(activity_name)
            total_activities_count+=1
        else:
            #print("activity start: {}".format(shutdown_sched_4[['Activities','START_plan']].iloc[act]))
            break
    
    equipment_percent_progress = round(100*count/total_activities_count, 2)
    equipment_percent_complte[eqpmnt_index] = equipment_percent_progress
    #print(eq+ " count: {}. total_prg_to_H_over: {}, tot_act_count: {}".format(count, equipment_percent_progress, total_activities_count))
    
df_prcnt_progress = pd.DataFrame(equipment_percent_complte)
df_prcnt_progress['equipment'] = List_equiments_rshp
df_prcnt_progress = df_prcnt_progress.rename(columns={0:'Percent_Progress_to_HandOver', 'equipment':'Equipment_Tag'})
df_prcnt_progress = df_prcnt_progress[['Equipment_Tag','Percent_Progress_to_HandOver']]
"""


In [ ]:
# spare code
"""
diff = reference_datetime - parse(str(time_start))
diff_hours = diff.days*24 + diff.seconds/3600
print(diff_hours)
"""

In [ ]:
# Spare code
"""
df_Shutdown_date_range = pd.DataFrame(Shutdown_date_range) # Create dataframe from Shutdown_date_range array
df_prcnt_progress_transposed = df_prcnt_progress.T # Transpose the df_prcnt_progress dataframe
equiment_list = df_prcnt_progress_transposed.iloc[-1] # get the list of equipments by slicing the last row of the df_prcnt_progress_transposed dataframe
df_prcnt_progress_transposed = df_prcnt_progress_transposed.drop('equipment')  # drop the row with equipment
df_prcnt_progress_transposed['DATETIME'] = df_Shutdown_date_range[0] # Create a column called the DATETIME using the values from df_Shutdown_date_range
df_prcnt_progress_transposed = df_prcnt_progress_transposed.set_index('DATETIME') # set the DATETIME as the index
df_prcnt_progress_transposed.columns = equiment_list # rename the columns with the equipment list using the equipment_list variable


# replace the values in the df_prcnt_progress_transposed to a range 0 to 100
for col in range(len(df_prcnt_progress_transposed.columns)):
    column = df_prcnt_progress_transposed.columns[col]
    df_prcnt_progress_transposed[column] = np.where(df_prcnt_progress_transposed[column]>=100, 100, np.where(df_prcnt_progress_transposed[column]<0, 0, df_prcnt_progress_transposed[column]))

columns_dezn = ['De-Zn Feedline', '105TK01', '105TK02', '105TK03', '105TK05',
                '105SP01/02', '105PU01ABCDEFG', '105FT01A', '105FT01B', '105FT01C',
                '105FT01D', '105FT01E', '105FT01F', '105TK13', '105TK08']

columns_MS = ['106TK01', 'MS FEEDLINE', '106VE01', '106VE02', '106VE03', 
              '106VE04', '106VE05', '106HX02', '106VB01AB', '106CP01AB', 
              '106TH01', '106FT02A', '106FT02B', '106TK08', '106TK04',
              '106TK05', '106TK18', '106TK14', '106SP02', 'MS Common Vent/ 106SP01',
              '106SR00/02\n106SP03/04', '106SR03', '106SR04', '106TK15/ NaOH header line']

columns_108_chem = [ '108TK01', '108TK04','24" SPT tie in', 'VSLC RHL Line', '112TK00',
                    '112TK01', '114HP/FD01', '114TK01', '114TK02', '114 Pumps', 
                    '115TK01','115TK02']
columns_111_ls = ['111 Belt Conveyors\n111VS01\n111CR01AB', '111ML01','111TK01', '111PU01/\n111CY01', '111TK02', 
                  '530CT31', '525NG01']

MS_progress_plan = np.sum(df_prcnt_progress_transposed[columns_MS], axis=1)/len(columns_MS)
DeZn_progress_plan = np.sum(df_prcnt_progress_transposed[columns_dezn], axis=1)/len(columns_dezn)
FNTRL_chem_progress_plan = np.sum(df_prcnt_progress_transposed[columns_108_chem], axis=1)/len(columns_108_chem)
LS_CT_progress_plan = np.sum(df_prcnt_progress_transposed[columns_111_ls], axis=1)/len(columns_111_ls)

df_prcnt_progress_transposed['MS_progress_plan'] = MS_progress_plan
df_prcnt_progress_transposed['DeZn_progress_plan'] = DeZn_progress_plan
df_prcnt_progress_transposed['FNTRL_chem_progress_plan'] = FNTRL_chem_progress_plan
df_prcnt_progress_transposed['LS_CT_progress_plan'] = LS_CT_progress_plan
"""